In [95]:
from datetime import datetime, timedelta
import pandas as pd

# Define the start and end dates for the year
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# Create an empty list to store the dates
date_list = []
date_list_str = []

# Use a loop to generate dates within the specified range
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date) 
    date_list_str.append(current_date.strftime("%m/%d/%Y")) 
    current_date += timedelta(days=1)

# Create a DataFrame with the dates
date_df = pd.DataFrame(date_list, columns=['DateTime'])
date_df['Date'] = date_list_str



In [96]:
# Add a column for the day of the week
date_df['Day_of_Week'] = date_df['DateTime'].apply(lambda x: x.strftime('%A'))
date_df['Day_of_Week_Index'] = date_df['DateTime'].dt.weekday

def check_weekend(date):
    day_of_week = date.weekday()
    return day_of_week >= 0 and day_of_week <= 4

date_df['Weekday?'] = date_df['DateTime'].apply(check_weekend)


In [97]:
EOM = ["01/31/2023", "02/28/2023", "03/31/2023", "04/30/2023", "05/31/2023", "06/30/2023", "07/31/2023", "08/31/2023", "09/30/2023", "10/31/2023", "11/30/2023", "12/31/2023"]

In [98]:
# Function to check if a date is in the EOM list
def check_eom(date):
    if date.strftime('%m/%d/%Y') in EOM:
        return True
    else:
        return False
    
def eom_weekend(date):
    if date.weekday() >=5:
        

# Apply the check_eom function to each row in the DataFrame and create a new 'EOM?' column
date_df['EOM?'] = date_df['DateTime'].apply(check_eom)

In [99]:
HOLIDAYS = ["01/17/2023", "02/21/2023", "05/30/2023", "07/05/2023", "09/05/2023", "11/24/2023", "12/26/2023", "01/02/2024"]

In [100]:
# Function to check if a date is the day after a Holiday
def check_day_after_hol(date):
    if date.strftime('%m/%d/%Y') in HOLIDAYS:
        return True
    else:
        return False
    

# Apply the check_day_after_hol function to each row in the DataFrame and create a new 'Holiday?' column
date_df['Holiday?'] = date_df['DateTime'].apply(check_day_after_hol)

In [ ]:
# Function to determine the File Date based on EOM, Holiday and/or Weekend

def determine_file_date(date):
    file_date = date + timedelta(days=1)
    print(date)
    print(file_date)
    flag = False
    while not flag:
        fd_dow = date_df.loc[date_df['DateTime'] == file_date, 'Weekday?']
        # print(fd_dow)
        fd_holiday = date_df.loc[date_df['DateTime'] == file_date, 'Holiday?']
        # print(fd_holiday)
        fd_eom = date_df.loc[date_df['DateTime'] == file_date, 'EOM?']
        # print(fd_eom)
        # Check the conditions
        if fd_dow.all() and not fd_holiday.any() and not fd_eom.any():
            return file_date  # Conditions met, return the file_date
        
        # Conditions not met, add 1 day to file_date and continue
        file_date += timedelta(days=1)
        print(file_date)  # For debugging purposes, to see how the loop progresses

date_df['File Date'] = date_df['DateTime'].apply(determine_file_date)

In [ ]:
# After Cutoff Function to determine the File Date based on EOM, Holiday and/or Weekend 

def determine_file_date(date):
    if date.weekday() == 4:
        file_date = date + timedelta(days=4)
    else:
        file_date = date + timedelta(days=2)
    print(date)
    print(file_date)
    flag = False
    while not flag:
        fd_dow = date_df.loc[date_df['DateTime'] == file_date, 'Weekday?']
        # print(fd_dow)
        fd_holiday = date_df.loc[date_df['DateTime'] == file_date, 'Holiday?']
        # print(fd_holiday)
        fd_eom = date_df.loc[date_df['DateTime'] == file_date, 'EOM?']
        # print(fd_eom)
        # Check the conditions
        if fd_dow.all() and not fd_holiday.any() and not fd_eom.any():
            return file_date  # Conditions met, return the file_date
        
        # Conditions not met, add 1 day to file_date and continue
        file_date += timedelta(days=1)
        print(file_date)  # For debugging purposes, to see how the loop progresses

date_df['After Cutoff'] = date_df['DateTime'].apply(determine_file_date)

In [103]:
date_df['Formatted FD'] = date_df['File Date'].apply(lambda x: x.strftime('%m/%d/%Y'))
date_df['Formatted Cutoff'] = date_df['After Cutoff'].apply(lambda x: x.strftime('%m/%d/%Y'))
columns_to_drop = ['DateTime', 'File Date', 'After Cutoff']
date_df.drop(columns=columns_to_drop, inplace=True)

date_df.to_csv("Dates.csv")